#### Import Necessari

In [7]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score

#### Caricamento Dataset

In [8]:
train1 = pd.read_json('dataset/Dataset20-21.json');
train2 = pd.read_json('dataset/Dataset21-22.json');
trainSet = pd.concat([train1, train2], ignore_index=True)
validationSet = pd.read_json('dataset/Dataset22-23.json');
testSet = pd.read_json('dataset/Dataset19-20.json');
trainSet.loc[trainSet["risultato"] == "X", "risultato"] = 3
trainSet.loc[trainSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "X", "risultato"] = 3

In [9]:
combined_data = pd.concat([trainSet, validationSet], axis=0, ignore_index=True)

In [10]:
X_combined = combined_data.drop("risultato", axis=1)
y_combined = combined_data["risultato"]

X_test = testSet.drop("risultato", axis=1)
y_test = testSet["risultato"]

In [11]:
X_combined_noName = X_combined
X_combined_noName = X_combined_noName.drop("squadraa", axis=1)
X_combined_noName = X_combined_noName.drop("squadrab", axis=1)
X_combined_noName = X_combined_noName.astype(float)
y_combined=y_combined.astype(float)

X_test_noName = X_test
X_test_noName = X_test_noName.drop("squadraa", axis=1)
X_test_noName = X_test_noName.drop("squadrab", axis=1)
X_test_noName = X_test_noName.astype(float)
#y_combined=y_combined.astype(float)

#### Inizio Codice training

In [12]:
svm_classifier = SVC(kernel='rbf') # OvO per la classificazione multiclasse
svm_classifier = OneVsOneClassifier(svm_classifier)

In [13]:
mask = ~np.isnan(y_combined)
y_combined = y_combined[mask]
X_combined_noName = X_combined_noName[mask] #Rimozione righe con NaN Values

from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(f_regression, k=5)
X_combined_noName = selector.fit_transform(X_combined_noName, y_combined)
X_test_noName = selector.transform(X_test_noName)


cross_val_scores = cross_val_score(svm_classifier, X_combined_noName, y_combined, cv=10, scoring='accuracy')

In [14]:
svm_classifier.fit(X_combined_noName, y_combined)
y_pred_test = svm_classifier.predict(X_test_noName)
accuracy_test = accuracy_score(y_test, y_pred_test)
finalPrecision = precision_score(y_test, y_pred_test, average='weighted', zero_division=1)
finalRecall = recall_score(y_test, y_pred_test, average='weighted', zero_division=1)
print(f"Accuracy: {accuracy_test}")
print("Accuratezza media della cross-validation:", cross_val_scores.mean())
print(f"Precision: {finalPrecision}")
print(f"Recall: {finalRecall}")


Accuracy: 0.4842105263157895
Accuratezza media della cross-validation: 0.49688712932774404
Precision: 0.6039770478828651
Recall: 0.4842105263157895
